# SVM - Climate Sentiment Multiclass Classification
## CS522 Project

### Dataset: 
https://www.kaggle.com/code/luiskalckstein/climate-sentiment-multiclass-classification

### Imports

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.svm import LinearSVC,SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from Common.DataCenter import data_center
from Common.LSI import SKLearnLSA
%matplotlib inline
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

time: 96.7 µs (started: 2022-04-07 22:52:09 +08:00)


### Text preprocessing

In [20]:
# parameter: original X of training set and test set
# return:  vectorised X of training set and test set
def text_preprocessing(X_train, X_test):
    # Convert texts to vectors
    vectorizer = CountVectorizer(min_df = 1, stop_words = 'english')
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    X_train_vec = X_train_vec.astype(float)
    X_test_vec = X_test_vec.astype(float)


    return X_train_vec, X_test_vec


time: 889 µs (started: 2022-04-07 23:00:47 +08:00)


### One-hot encoding, convert the labels to vectors (4 x 1) each

In [3]:
# parameter: original y of training set, original y of test set
# return:  encoded y of training set and test set
def one_hot_encoding(y_train, y_test):
    mlb          = MultiLabelBinarizer()
    y_train_vec  = mlb.fit_transform(map(str, y_train))
    y_test_vec   = mlb.transform(map(str, y_test))
    return y_train_vec, y_test_vec


time: 212 µs (started: 2022-04-07 22:52:09 +08:00)


### Run SVM and evaluate the results

In [32]:
# parameter:  vectorised X and encoded y of training set and test set
def evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec):
    # Run SVM - fit and predict
    SVM             = OneVsRestClassifier(LinearSVC(dual=False, class_weight='balanced'), n_jobs=-1)
    #SVM = OneVsRestClassifier(SVC(gamma='auto'), n_jobs=-1)
    SVM.fit(X_train_vec, y_train_vec)
    prediction      = SVM.predict(X_test_vec)

    # Evaluate the results
    macro_f1        = f1_score(y_test_vec, prediction, average='macro')
    weighted_f1     = f1_score(y_test_vec, prediction, average='weighted')
    macro_precision = precision_score(y_test_vec, prediction, average='macro')
    macro_recall    = recall_score(y_test_vec, prediction, average='macro')

    return macro_f1, weighted_f1, macro_precision, macro_recall


time: 1.27 ms (started: 2022-04-07 23:03:51 +08:00)


### Do an experiment

In [5]:
# Parameter: original X,y of training set and test set
def do_experiment(X_train, y_train, X_test, y_test):
    # Convert texts to vectors
    X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
    y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

    # Run SVM and evaluate the results
    macro_f1, weighted_f1, macro_precision, macro_recall = \
        evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

    # Show the indicators
    print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
          (macro_f1, weighted_f1, macro_precision, macro_recall))
    return X_train_vec


time: 317 µs (started: 2022-04-07 22:52:09 +08:00)


### Main entry

**Load the database and split it into training set, test set, noisy set, validation set**

In [6]:
dc = data_center("twitter_sentiment_data_clean.csv", test_size=8000, noisy_size=8000, validation_size=5000)

print("####################################################")
print("Total data size: ",       dc.get_len())
print("Total train data size: ", dc.get_train_len())
print("Total test data size: ",  dc.get_test_len())

####################################################
Total data size:  40908
Total train data size:  19908
Total test data size:  8000
time: 82.4 ms (started: 2022-04-07 22:52:09 +08:00)


**Get the test set for evaluation**

In [7]:
X_test, y_test = dc.get_test()

time: 3.65 ms (started: 2022-04-07 22:52:09 +08:00)


**Run experiments with different training sets, and use the same test set.**

In [34]:
print("-----------------------------------------------")
for size in [2000, 2500, 4000, 5000, 7500, 10000]:
    # Get a training set without noisy data
    X_train, y_train = dc.get_train(size)
    print("Training set size: %d samples (%.1f%%): " % (len(X_train), len(y_train)/dc.get_train_len()*100))

    # Do an experiment
    do_experiment(X_train, y_train, X_test, y_test)

print("-----------------------------------------------")
xtrainvec = None
for size in [(2000, 500), (4000, 1000), (7500, 2500)]:
    # Get a noisy training set
    X_train, y_train = dc.get_train_with_noisy(size[0], size[1])
    print("Noisy training set size: %d samples (%d original, %d noisy)" % (len(y_train), size[0], size[1]))

    # Do an experiment
    xtrainvec = do_experiment(X_train, y_train, X_test, y_test)

-----------------------------------------------
Training set size: 2000 samples (10.0%): 
 macro_f1: 0.4865 , weighted_f1: 0.5612, macro_precision: 0.6008, macro_recall: 0.4291
Training set size: 2500 samples (12.6%): 
 macro_f1: 0.4982 , weighted_f1: 0.5702, macro_precision: 0.6019, macro_recall: 0.4437
Training set size: 4000 samples (20.1%): 
 macro_f1: 0.5427 , weighted_f1: 0.6063, macro_precision: 0.6168, macro_recall: 0.4960
Training set size: 5000 samples (25.1%): 
 macro_f1: 0.5555 , weighted_f1: 0.6156, macro_precision: 0.6133, macro_recall: 0.5139
Training set size: 7500 samples (37.7%): 
 macro_f1: 0.5741 , weighted_f1: 0.6307, macro_precision: 0.6270, macro_recall: 0.5354
Training set size: 10000 samples (50.2%): 
 macro_f1: 0.5921 , weighted_f1: 0.6445, macro_precision: 0.6346, macro_recall: 0.5595
-----------------------------------------------
Noisy training set size: 2499 samples (2000 original, 500 noisy)
 macro_f1: 0.4225 , weighted_f1: 0.4889, macro_precision: 0.4984

In [33]:
X_train, y_train = dc.get_train(2000)
# Convert texts to vectors
X_train_vec, X_test_vec = text_preprocessing(X_train, X_test)
y_train_vec, y_test_vec = one_hot_encoding(y_train, y_test)

# Run SVM and evaluate the results
macro_f1, weighted_f1, macro_precision, macro_recall = \
    evaluate_SVM(X_train_vec, y_train_vec, X_test_vec, y_test_vec)

# Show the indicators
print(" macro_f1: %.4f , weighted_f1: %.4f, macro_precision: %.4f, macro_recall: %.4f" %
        (macro_f1, weighted_f1, macro_precision, macro_recall))

 macro_f1: 0.4865 , weighted_f1: 0.5612, macro_precision: 0.6008, macro_recall: 0.4291
time: 351 ms (started: 2022-04-07 23:03:56 +08:00)


In [ ]:
import pandas as pd
df = pd.DataFrame([1,2,3])